# Belajar Mengonsumsi API dengan Python

Notebook ini membimbing kamu memahami cara kerja API (Application Programming Interface) menggunakan Python dan library `requests`. Semua penjelasan disajikan dalam bahasa Indonesia dan diakhiri dengan contoh mini-proyek.

## Tujuan Pembelajaran
- Mengetahui konsep dasar API dan HTTP request.
- Menggunakan `requests` untuk melakukan `GET` dan `POST`.
- Mengirim parameter query serta payload JSON.
- Menangani error dan menyusun helper function.
- Mengolah data hasil API untuk studi kasus kecil.

> Kita akan menggunakan layanan publik [JSONPlaceholder](https://jsonplaceholder.typicode.com) yang gratis untuk latihan dan tidak membutuhkan API key.

## 1. Persiapan & Pengecekan Modul
Pastikan modul `requests` sudah terpasang (biasanya sudah ada di lingkungan standar). Jalankan sel di bawah ini untuk mengimpor modul dan melihat versinya.

In [ ]:
import requests
from pprint import pprint

print(f"Versi requests: {requests.__version__}")

## 2. Sekilas Tentang API dan HTTP
API memungkinkan aplikasi saling berbicara melalui protokol HTTP. Beberapa komponen penting:

- **Endpoint**: URL tujuan (misal `/posts/1`).
- **Method**: GET (ambil data), POST (kirim data baru), PUT/PATCH (ubah), DELETE (hapus).
- **Status code**: 200 artinya sukses, 404 data tidak ditemukan, 500 error server, dll.
- **Header & Body**: metadata permintaan dan isi pesan.

Kita akan fokus pada GET dan POST karena paling umum saat mulai belajar.

## 3. Mengambil Data dengan `GET`
Contoh pertama mengambil 1 posting dari endpoint `/posts/1`. Kita akan melihat `status_code` dan hasil JSON yang dikembalikan.

In [ ]:
import requests
from pprint import pprint

url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url)

print("Status:", response.status_code)
data = response.json()
pprint(data)

### Analisis Hasil
- Jika status 200, berarti permintaan berhasil.
- `response.json()` otomatis mengubah teks JSON menjadi dictionary Python.
- Dari data di atas kita bisa mengambil `title`, `body`, atau `userId` untuk kebutuhan aplikasi.

## 4. Menggunakan Query Parameter
API sering menerima parameter tambahan. Misalnya kita ingin mengambil semua komentar (`/comments`) untuk `postId=1`. Kita bisa memanfaatkan argumen `params` di `requests.get` agar parameter otomatis dirangkai ke URL.

In [ ]:
import requests
from pprint import pprint

params = {"postId": 1}
response = requests.get("https://jsonplaceholder.typicode.com/comments", params=params)
comments = response.json()

print(f"Jumlah komentar yang diterima: {len(comments)}")
print("Contoh 2 komentar pertama:")
pprint(comments[:2])

## 5. Membuat Helper Function untuk Permintaan API
Agar kode lebih rapi, kita buat fungsi `fetch_json` yang menerima path endpoint, parameter, method, dan payload. Fungsi ini juga menangani error (mis. timeout atau status non-200) dengan blok `try/except`.

In [ ]:
import requests

BASE_URL = "https://jsonplaceholder.typicode.com"


def fetch_json(path, params=None, method="GET", payload=None):
    '''Mengirim permintaan HTTP dan mengembalikan hasil JSON.'''
    url = f"{BASE_URL}{path}"
    try:
        if method.upper() == "GET":
            resp = requests.get(url, params=params, timeout=10)
        elif method.upper() == "POST":
            resp = requests.post(url, json=payload, timeout=10)
        else:
            raise ValueError("Metode belum didukung dalam contoh ini")

        resp.raise_for_status()
        return resp.json()
    except requests.RequestException as err:
        print("Terjadi kesalahan saat mengakses API:", err)
        return None

## 6. Mengambil Banyak Data dan Menampilkannya
Dengan helper di atas, kita dapat mengambil beberapa posting sekaligus dan menampilkannya dalam format sederhana.

In [ ]:
posts = fetch_json("/posts", params={"_limit": 5})
if posts:
    for post in posts:
        print(f"ID {post['id']:>2} | User {post['userId']} | Judul: {post['title'][:40]}...")
else:
    print("Data tidak tersedia.")

## 7. Mengirim Data Baru dengan `POST`
JSONPlaceholder mengizinkan kita mensimulasikan pembuatan data lewat metode POST. Walaupun data tidak benar-benar tersimpan permanen, responsnya berguna untuk latihan.

In [ ]:
new_post_payload = {
    "title": "Belajar API dengan Python",
    "body": "Contoh konten yang akan dikirim ke server.",
    "userId": 99,
}

created_post = fetch_json("/posts", method="POST", payload=new_post_payload)
print("Respons dari server:")
print(created_post)

### Catatan Penting
- Untuk API nyata, biasanya kita perlu API key/token. Letakkan token tersebut di header (`headers={"Authorization": "Bearer ..."}`).
- Saat membuat user interface, selalu validasi input sebelum dikirim.
- Jangan lupa membaca dokumentasi API untuk mengetahui parameter wajib, batas rate-limit, dan format respons.

## 8. Mini Proyek: Ringkasan To-Do Pengguna
Sebagai latihan akhir, kita akan mengambil semua data to-do milik pengguna tertentu, menghitung jumlah tugas selesai/belum, dan menampilkan sebagian daftar.

In [ ]:
def ringkasan_todo(user_id: int):
    todos = fetch_json("/todos", params={"userId": user_id})
    if not todos:
        print("Tidak bisa mengambil data to-do.")
        return

    selesai = [todo for todo in todos if todo["completed"]]
    belum = [todo for todo in todos if not todo["completed"]]

    print(f"User ID: {user_id}")
    print(f"Total tugas : {len(todos)}")
    print(f"Selesai      : {len(selesai)}")
    print(f"Belum selesai: {len(belum)}
")

    print("Contoh daftar tugas (maks 5 item):")
    for todo in todos[:5]:
        status = "✓" if todo["completed"] else "✗"
        print(f"[{status}] #{todo['id']:>3} - {todo['title']}")


ringkasan_todo(1)

## 9. Langkah Lanjutan
- Coba ubah kode mini proyek agar menerima input ID pengguna dari pengguna (`input()` atau widget GUI).
- Simpan data hasil API ke file CSV/JSON menggunakan modul `csv` atau `json`.
- Eksplorasi API publik lainnya seperti OpenWeatherMap, TheMovieDB, atau API kampusmu sendiri.

Selamat! Kamu sudah mempelajari dasar mengonsumsi API menggunakan Python.